<a href="https://colab.research.google.com/github/ajaysaikiran2208/MLCONCEPTS/blob/main/Market_Basket_Analysis_using_assocition_rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Market Basket Analysis using assocition rules

Association rules analysis is a technique to uncover how items are associated to each other. There are three common ways to measure association.


Measure 1: Support. This says how popular an itemset is, it is number of times appear in total number of transaction. in other word we say frequency of item.


Measure 2: Confidence. This says how likely item Y is purchased when item X is purchased, expressed as {X -> Y}. This is measured by the proportion of transactions with item X, in which item Y also appears.


Measure 3: Lift. it is ratio of expected confidance to observed confidance. it is described as confidance of Y when item X was already known(x/y) to the confidance of Y when X item is unknown. in other words confidance of Y w.r.t. x and confiadnce of Y without X (means both are independent to each other).

#Metrics

support = occurance of item / total no of transaction.


confidance = support ( X Union Y) / support(X).


lift = support (X Union Y)/ support(X) * support(Y) .




In [2]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-cp37-none-any.whl size=5975 sha256=ff1af3728b790fe1a1ada82a2318ec23b98e84a6679ce222519197418bcc768e
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [13]:
import pandas as pd
import numpy as np
from apyori import apriori

In [5]:
df=pd.read_csv("/content/Market_Basket_Optimisation.csv",header=None)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.size

150020

In [19]:
transactions = []
for i in range(0,len(df)):
    transactions.append([str(df.values[i,j]) for j in range(0,20) if str(df.values[i,j])!='0'])

In [20]:
transactions[0]

['shrimp',
 'almonds',
 'avocado',
 'vegetables mix',
 'green grapes',
 'whole weat flour',
 'yams',
 'cottage cheese',
 'energy drink',
 'tomato juice',
 'low fat yogurt',
 'green tea',
 'honey',
 'salad',
 'mineral water',
 'salmon',
 'antioxydant juice',
 'frozen smoothie',
 'spinach',
 'olive oil']

In [21]:
#Call apriori function which requires minimum support, confidance and lift, min length is combination of item default is 2".
rules = apriori(transactions,min_support=0.003,min_confidance=0.2,min_lift=3,min_length=2)

In [22]:
#it generates a set of rules in a generator file...
rules

<generator object apriori at 0x7f0c3a154a50>

In [23]:
# all rules need to be converted in a list..
Results = list(rules)
Results

[RelationRecord(items=frozenset({'cottage cheese', 'brownies'}), support=0.0034662045060658577, ordered_statistics=[OrderedStatistic(items_base=frozenset({'brownies'}), items_add=frozenset({'cottage cheese'}), confidence=0.10276679841897232, lift=3.225329518580382), OrderedStatistic(items_base=frozenset({'cottage cheese'}), items_add=frozenset({'brownies'}), confidence=0.10878661087866107, lift=3.2253295185803816)]),
 RelationRecord(items=frozenset({'light cream', 'chicken'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'chicken'}), items_add=frozenset({'light cream'}), confidence=0.07555555555555556, lift=4.843950617283951), OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'escalope', 'mushroom cream sauce'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'escalope'}),

In [24]:
#convert result in a dataframe for further operation...
df_results = pd.DataFrame(Results)

In [25]:
# as we see order statistics itself a list so need to be converted in proper format..
df_results.head()

,items,support,ordered_statistics
0,"(cottage cheese, brownies)",0.003466,"[((brownies), (cottage cheese), 0.102766798418..."
1,"(light cream, chicken)",0.004533,"[((chicken), (light cream), 0.0755555555555555..."
2,"(escalope, mushroom cream sauce)",0.005733,"[((escalope), (mushroom cream sauce), 0.072268..."
3,"(escalope, pasta)",0.005866,"[((escalope), (pasta), 0.07394957983193277, 4...."
4,"(fresh bread, tomato juice)",0.004266,"[((fresh bread), (tomato juice), 0.09907120743..."


In [26]:
#keep support in a separate data frame so we can use later.. 
support = df_results.support

In [27]:
#all four empty list which will contain lhs, rhs, confidance and lift respectively.

first_values = []
second_values = []
third_values = []
fourth_value = []

# loop number of rows time and append 1 by 1 value in a separate list.. first and second element was frozenset which need to be converted in list..
for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])

In [28]:
#convert all four list into dataframe for further operation..
lhs = pd.DataFrame(first_values)
rhs= pd.DataFrame(second_values)
confidance=pd.DataFrame(third_values,columns=['Confidance'])
lift=pd.DataFrame(fourth_value,columns=['lift'])

In [56]:

#concat all list together in a single dataframe
df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final

,0,1,0,1,2,3,support,Confidance,lift
0,brownies,None,cottage cheese,None,None,None,0.003466,0.102767,3.225330
1,chicken,None,light cream,None,None,None,0.004533,0.075556,4.843951
2,escalope,None,mushroom cream sauce,None,None,None,0.005733,0.072269,3.790833
3,escalope,None,pasta,None,None,None,0.005866,0.073950,4.700812
4,fresh bread,None,tomato juice,None,None,None,0.004266,0.099071,3.259356
...,...,...,...,...,...,...,...,...,...
183,pancakes,ground beef,mineral water,nan,spaghetti,None,0.003066,0.211009,3.532991
184,ground beef,None,mineral water,tomatoes,nan,spaghetti,0.003066,0.031208,3.344117
185,olive oil,None,milk,mineral water,nan,spaghetti,0.003333,0.050607,3.216994
186,milk,mineral water,shrimp,nan,spaghetti,None,0.003066,0.063889,3.014029


In [57]:
df_final.fillna(value='0', inplace=True)

In [58]:
#set column name
df_final.columns = ['lhs',1,2,'rhs',3,4,'support','confidance','lift']

In [59]:
df_final

,lhs,1,2,rhs,3,4,support,confidance,lift
0,brownies,0,cottage cheese,0,0,0,0.003466,0.102767,3.225330
1,chicken,0,light cream,0,0,0,0.004533,0.075556,4.843951
2,escalope,0,mushroom cream sauce,0,0,0,0.005733,0.072269,3.790833
3,escalope,0,pasta,0,0,0,0.005866,0.073950,4.700812
4,fresh bread,0,tomato juice,0,0,0,0.004266,0.099071,3.259356
...,...,...,...,...,...,...,...,...,...
183,pancakes,ground beef,mineral water,nan,spaghetti,0,0.003066,0.211009,3.532991
184,ground beef,0,mineral water,tomatoes,nan,spaghetti,0.003066,0.031208,3.344117
185,olive oil,0,milk,mineral water,nan,spaghetti,0.003333,0.050607,3.216994
186,milk,mineral water,shrimp,nan,spaghetti,0,0.003066,0.063889,3.014029


In [60]:
df_final['lhs'] = df_final['lhs']+str(", ")+df_final[1]+str(", ")+df_final[2]

In [61]:
df_final

,lhs,1,2,rhs,3,4,support,confidance,lift
0,"brownies, 0, cottage cheese",0,cottage cheese,0,0,0,0.003466,0.102767,3.225330
1,"chicken, 0, light cream",0,light cream,0,0,0,0.004533,0.075556,4.843951
2,"escalope, 0, mushroom cream sauce",0,mushroom cream sauce,0,0,0,0.005733,0.072269,3.790833
3,"escalope, 0, pasta",0,pasta,0,0,0,0.005866,0.073950,4.700812
4,"fresh bread, 0, tomato juice",0,tomato juice,0,0,0,0.004266,0.099071,3.259356
...,...,...,...,...,...,...,...,...,...
183,"pancakes, ground beef, mineral water",ground beef,mineral water,nan,spaghetti,0,0.003066,0.211009,3.532991
184,"ground beef, 0, mineral water",0,mineral water,tomatoes,nan,spaghetti,0.003066,0.031208,3.344117
185,"olive oil, 0, milk",0,milk,mineral water,nan,spaghetti,0.003333,0.050607,3.216994
186,"milk, mineral water, shrimp",mineral water,shrimp,nan,spaghetti,0,0.003066,0.063889,3.014029


In [63]:
df_final['rhs'] = df_final['rhs']+str(", ")+df_final[3]+str(", ")+df_final[4]

In [64]:
df_final

,lhs,1,2,rhs,3,4,support,confidance,lift
0,"brownies, 0, cottage cheese",0,cottage cheese,"0, 0, 0",0,0,0.003466,0.102767,3.225330
1,"chicken, 0, light cream",0,light cream,"0, 0, 0",0,0,0.004533,0.075556,4.843951
2,"escalope, 0, mushroom cream sauce",0,mushroom cream sauce,"0, 0, 0",0,0,0.005733,0.072269,3.790833
3,"escalope, 0, pasta",0,pasta,"0, 0, 0",0,0,0.005866,0.073950,4.700812
4,"fresh bread, 0, tomato juice",0,tomato juice,"0, 0, 0",0,0,0.004266,0.099071,3.259356
...,...,...,...,...,...,...,...,...,...
183,"pancakes, ground beef, mineral water",ground beef,mineral water,"nan, spaghetti, 0",spaghetti,0,0.003066,0.211009,3.532991
184,"ground beef, 0, mineral water",0,mineral water,"tomatoes, nan, spaghetti",nan,spaghetti,0.003066,0.031208,3.344117
185,"olive oil, 0, milk",0,milk,"mineral water, nan, spaghetti",nan,spaghetti,0.003333,0.050607,3.216994
186,"milk, mineral water, shrimp",mineral water,shrimp,"nan, spaghetti, 0",spaghetti,0,0.003066,0.063889,3.014029


In [65]:
df_final.drop(columns=[1,2,3,4],inplace=True)

In [67]:
df_final

,lhs,rhs,support,confidance,lift
0,"brownies, 0, cottage cheese","0, 0, 0",0.003466,0.102767,3.225330
1,"chicken, 0, light cream","0, 0, 0",0.004533,0.075556,4.843951
2,"escalope, 0, mushroom cream sauce","0, 0, 0",0.005733,0.072269,3.790833
3,"escalope, 0, pasta","0, 0, 0",0.005866,0.073950,4.700812
4,"fresh bread, 0, tomato juice","0, 0, 0",0.004266,0.099071,3.259356
...,...,...,...,...,...
183,"pancakes, ground beef, mineral water","nan, spaghetti, 0",0.003066,0.211009,3.532991
184,"ground beef, 0, mineral water","tomatoes, nan, spaghetti",0.003066,0.031208,3.344117
185,"olive oil, 0, milk","mineral water, nan, spaghetti",0.003333,0.050607,3.216994
186,"milk, mineral water, shrimp","nan, spaghetti, 0",0.003066,0.063889,3.014029
